In [31]:
using JuMP
using Gurobi
using Random
using Distributions

In [32]:
N=6 #No. of nodes (1 supplier + 5 DC's)

K=3 #No. of vehicles

Dk=900 #Capacity of vehicles

T=8 #Periods

h=0.025 #Holding cost/unit at each DC

C=[50000 5020 4880 2200 4860 7420] #Capacity limit for (1 supplier + 5 DC's) increased by 10 times

d=[0 0 0 0 0 0 0 0;
75.9 75.9 75.9 75.9 75.9 85.7 85.7 75.9;
62.9 94.8 94.8 94.8 94.8 68.65 36.75 62.9;
67.2 67.2 67.2 67.2 67.2 67.15 67.15 67.2;
102.3 156 156 156 156 119.05 65.35 102.3;
107.4 130.1 130.1 130.1 130.1 65.15 42.45 107.4
] #Demand for each day and (1 supplier + 5 DC's)

c=[
0 140 434 389 419 125;
140 0 300 455 400 97;
434 300 0 609 417 330;
389 455 609 0 256 358;
419 400 417 256 0 316;
125 97 330 358 316 0
] #Transportation cost

std=[
0 0 0 0 0 0 0 0;
11.87 11.87 11.87 11.87 11.87 14.69 14.69 11.87;
10.91 13.89 13.89 13.89 13.89 14.14 11.22 10.91;
11.27 11.27 11.27 11.27 11.27 12.86 12.86 11.27;
13.19 16.03 16.03 16.03 16.03 15.05 11.98 13.19;
13.04 14.00 14.00 14.00 14.00 11.08 9.84 13.04
]# Standard deviation of Demand


I0=[0 98.8 153.9 42.4 23.4 85.7] #Initial inventory
alpha=0.913 # Critical fractile
za=quantile.(Normal(0,1),alpha) #Safety factor
d_new = d + za.*std; #Demand to be satisfied

In [33]:
model=Model(Gurobi.Optimizer)
@variable(model, x[i=1:N,j=1:N,k=1:K,t=1:T], Bin); #If truck k leaves DC i and goes to DC j
@variable(model, y[i=1:N,k=1:K,t=1:T], Bin); #If truck k arrives at DC i
@variable(model, z[j=1:N,k=1:K,t=1:T] >= 0); #Load of truck k when arriving at node j
@variable(model, I[i=1:N,t=0:T] >= 0); #Inventory level at node i till end of period t
@variable(model, q[i=1:N,k=1:K,t=1:T] >= 0); #Quantity delivered to customer i by truck k at start of period t

@objective(model, Min, sum(h*I[i,t] for i=2:N, t=1:T) + sum(c[i,j]*x[i,j,k,t]  for i=1:N, j=1:N, k=1:K, t=1:T))

@constraint(model, [i=2:N], I[i,0] == I0[i]) #Initializing inventory level

@constraint(model, [i=2:N,t=1:T], I[i,t-1] + sum(q[i,k,t] for k=1:K) == I[i,t] + d_new[i,t]); #Balance constraint for each DC

@constraint(model, [i=2:N,t=1:T], I[i,t-1] + sum(q[i,k,t] for k=1:K) <= C[i]); #Capacity constraint for each DC

@constraint(model, [i=2:N,k=1:K,t=1:T], q[i,k,t] <= C[i]*y[i,k,t]); #Constraint which ensures y is 1 when they receive a quantity

@constraint(model, [k=1:K,t=1:T], sum(q[i,k,t] for i=2:N) <= Dk*y[1,k,t]); #If truck delivers from supplier, it has to activate the supplier

@constraint(model, [h=1:N,k=1:K,t=1:T], sum(i == h ? 0 : x[i,h,k,t] for i = 1:N) == y[h,k,t]); #If truck visits DC

@constraint(model, [h=1:N,k=1:K,t=1:T], sum(j == h ? 0 : x[h,j,k,t] for j = 1:N) == y[h,k,t]); #If truck leaves DC

@constraint(model, [i=2:N,j=2:N,k=1:K,t=1:T], z[i,k,t] - d_new[i,t] >= z[j,k,t] - (1-x[i,j,k,t])*sum(d_new)); #To avoid subtours automatically



Set parameter Username
Academic license - for non-commercial use only - expires 2023-03-06


In [34]:
optimize!(model)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1117 rows, 1350 columns and 4037 nonzeros
Model fingerprint: 0x22214373
Variable types: 342 continuous, 1008 integer (1008 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [3e-02, 6e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 7e+03]
Found heuristic solution: objective 10128.379455
Presolve removed 162 rows and 213 columns
Presolve time: 0.02s
Presolved: 955 rows, 1137 columns, 3758 nonzeros
Variable types: 273 continuous, 864 integer (864 binary)

Root relaxation: objective 2.025059e+03, 697 iterations, 0.02 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 2025.05873    0   91 10128.3795 2025.05873  80.0%     -    0s
H    0     0          

In [35]:

if termination_status(model) == MOI.OPTIMAL
   println("RESULTS:")
   println("Objective = $(objective_value(model))")
else
   println("No solution")
end

println();
    for k=1:K
        has_route = false;
        print("Truck:$(value(k))\n")
        for t=1:T
            print("Day:$(value(t))\n")
            for i=1:N
                for j=1:N
                    if (value(x[i,j,k,t]) == 1)
                        print(j,"->",i,"\n")
                        print("$(value(q[i,k,t]))\n")
                        has_route = true;
                    end
                end
            end
        end
        if has_route
            println();
        end
    end

for i=1:N
    for k=1:K
        for t=1:T
            if (value(q[i,k,t]) > 0)
                println(" Node ",i," Truck ",k," Day ", t," Delivery ",value(q[i,k,t]))
            end
        end
    end
end


for t=1:T
    println("Day", t,)
    for i=2:N
        println("Node ",i," Inventory Level = ",value(I[i,t]))
    end
end


RESULTS:
Objective = 3561.1655518178823

Truck:1
Day:1
6->1
0.0
1->6
897.1246627780527
Day:2
2->1
0.0
3->2
269.3472911524555
1->3
596.1712024642434
Day:3
5->1
0.0
1->5
874.7541550478841
Day:4
Day:5
Day:6
Day:7
Day:8

Truck:2
Day:1
Day:2
Day:3
Day:4
Day:5
2->1
0.0
1->2
395.41466103661696
Day:6
Day:7
Day:8

Truck:3
Day:1
4->1
0.0
5->4
621.9922526572986
1->5
274.62350142112103
Day:2
Day:3
Day:4
Day:5
Day:6
Day:7
Day:8

 Node 2 Truck 1 Day 2 Delivery 269.3472911524555
 Node 2 Truck 1 Day 4 Delivery 3.439026841078885e-12
 Node 2 Truck 2 Day 5 Delivery 395.41466103661696
 Node 2 Truck 2 Day 6 Delivery 9.094947017729282e-13
 Node 3 Truck 1 Day 2 Delivery 596.1712024642434
 Node 3 Truck 1 Day 4 Delivery 5.684341886080802e-14
 Node 4 Truck 3 Day 1 Delivery 621.9922526572986
 Node 5 Truck 1 Day 3 Delivery 874.7541550478841
 Node 5 Truck 2 Day 4 Delivery 9.094947017729282e-13
 Node 5 Truck 3 Day 1 Delivery 274.62350142112103
 Node 6 Truck 1 Day 1 Delivery 897.1246627780527
Day1
Node 2 Inventory L

In [36]:
#Truck Capacity utilization
for k=1:K
    println("Truck ",k)
    for t=1:T
        println("on day ",t," utilizes ", value(sum(q[i,k,t] for i=1:N))/value(Dk))
    end
end

Truck 1
on day 1 utilizes 0.996805180864503
on day 2 utilizes 0.9616872151296654
on day 3 utilizes 0.9719490611643157
on day 4 utilizes 3.884300288821881e-15
on day 5 utilizes 0.0
on day 6 utilizes 0.0
on day 7 utilizes 0.0
on day 8 utilizes 0.0
Truck 2
on day 1 utilizes 0.0
on day 2 utilizes 0.0
on day 3 utilizes 0.0
on day 4 utilizes 1.010549668636587e-15
on day 5 utilizes 0.43934962337401884
on day 6 utilizes 1.010549668636587e-15
on day 7 utilizes 0.0
on day 8 utilizes 0.0
Truck 3
on day 1 utilizes 0.9962397267537997
on day 2 utilizes 0.0
on day 3 utilizes 0.0
on day 4 utilizes 0.0
on day 5 utilizes 0.0
on day 6 utilizes 0.0
on day 7 utilizes 0.0
on day 8 utilizes 0.0


In [37]:
#Capacity utilization of the DCs
for t=1:T
    println("Day", t,)
    for i=2:N
        println("Node ",i," Inventory Level = ",value(I[i,t])/value(C[i]))
    end
end


Day1
Node 2 Inventory Level = 0.0013472464565508493
Node 3 Inventory Level = 0.01560825029661615
Node 4 Inventory Level = 0.26448686705292496
Node 5 Inventory Level = 0.03658275469322113
Node 6 Inventory Level = 0.11559262379754694
Day2
Node 2 Inventory Level = 0.03666805688769443
Node 3 Inventory Level = 0.11447879638890786
Node 4 Inventory Level = 0.22697725562525942
Node 5 Inventory Level = 0.0
Node 6 Inventory Level = 0.09549390702721594
Day3
Node 2 Inventory Level = 0.01833402844384687
Node 3 Inventory Level = 0.091183112468035
Node 4 Inventory Level = 0.1894676441975938
Node 5 Inventory Level = 0.14340781218905954
Node 6 Inventory Level = 0.07539519025688499
Day4
Node 2 Inventory Level = 0.0
Node 3 Inventory Level = 0.06788742854716547
Node 4 Inventory Level = 0.1519580327699281
Node 5 Inventory Level = 0.10682505749583861
Node 6 Inventory Level = 0.05529647348655511
Day5
Node 2 Inventory Level = 0.060433832320418775
Node 3 Inventory Level = 0.044591744626292604
Node 4 Inventory 

In [38]:
Random.seed!(0);
counter=zeros(6,8)
#probability=zeros(6,8)
for r = 1:10000
    actual_demand = d + randn(6,8).*std;
    for i=1:N
        new_IL=I0[i];
        for t=1:T
            delivered=0;
            for k=1:K
                delivered = delivered + value(q[i,k,t]);
            end
            new_IL=new_IL+delivered-value(actual_demand[i,t]);
            fillrate=(max(value(actual_demand[i,t])+min(new_IL,0),0))/value(actual_demand[i,t]);
            if fillrate>0.95
                counter[i,t]=counter[i,t] + 1
            end
        end
    end
end
probability=counter/10000

6×8 Matrix{Float64}:
 0.0    0.0   0.0  0.0    0.0  0.0  0.0     0.0
 0.992  1.0   1.0  0.998  1.0  1.0  1.0     0.9997
 1.0    1.0   1.0  1.0    1.0  1.0  0.9996  1.0
 1.0    1.0   1.0  1.0    1.0  1.0  1.0     1.0
 1.0    0.99  1.0  1.0    1.0  1.0  1.0     1.0
 1.0    1.0   1.0  1.0    1.0  1.0  1.0     1.0

In [39]:
probability # Probability of meeting the fillrate target

6×8 Matrix{Float64}:
 0.0    0.0   0.0  0.0    0.0  0.0  0.0     0.0
 0.992  1.0   1.0  0.998  1.0  1.0  1.0     0.9997
 1.0    1.0   1.0  1.0    1.0  1.0  0.9996  1.0
 1.0    1.0   1.0  1.0    1.0  1.0  1.0     1.0
 1.0    0.99  1.0  1.0    1.0  1.0  1.0     1.0
 1.0    1.0   1.0  1.0    1.0  1.0  1.0     1.0

In [40]:
safety_stock=za.*std # Safety stock

6×8 Matrix{Float64}:
  0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0
 16.1368  16.1368  16.1368  16.1368  16.1368  19.9705  19.9705  16.1368
 14.8317  18.8829  18.8829  18.8829  18.8829  19.2228  15.2532  14.8317
 15.3211  15.3211  15.3211  15.3211  15.3211  17.4827  17.4827  15.3211
 17.9313  21.7922  21.7922  21.7922  21.7922  20.4599  16.2864  17.9313
 17.7274  19.0325  19.0325  19.0325  19.0325  15.0628  13.3771  17.7274

In [41]:
using DelimitedFiles

In [42]:
writedlm( "3.2b.csv",  safety_stock, ',')